# Profiling MNIST example with 3-conv. layer network

This example demonstrates the usage of `HandlersTimeProfiler`. The example uses MNIST dataset.

In [1]:
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

# A hack to fix the horizontal spill in large output
# ref: https://stackoverflow.com/a/59058418/6574605
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

## Loading MNIST

In [2]:
mnist_pwd = "data"
batch_size= 256

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

trainset = MNIST(mnist_pwd, train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)

testset = MNIST(mnist_pwd, train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size * 2, shuffle=False, num_workers=0)

## Model

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [5]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss, Accuracy
from ignite.contrib.handlers import ProgressBar, HandlersTimeProfiler

## Training Loss

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
criterion = nn.NLLLoss()
model = Net()
model.to(device)  # Move model before creating optimizer
optimizer = optim.SGD(model.parameters(), lr=3e-4, momentum=0.9)

In [7]:
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, metrics={"acc": Accuracy(), "loss": Loss(nn.NLLLoss())}, device=device)

# Attach handlers profiler
profiler = HandlersTimeProfiler()
profiler.attach(trainer)

# Init and attach progressbar
pbar = ProgressBar(persist=True)
pbar.attach(trainer, metric_names="all")

# Evaluate on each epoch using event handler
@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    evaluator.run(testloader)
    metrics = evaluator.state.metrics
    avg_accuracy = metrics["acc"]
    avg_nll = metrics["loss"]
    pbar.log_message(
        "Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
            engine.state.epoch, avg_accuracy, avg_nll
        )
    )

    pbar.n = pbar.last_print_n = 0

trainer.run(trainloader, max_epochs=10)


Validation Results - Epoch: 1  Avg accuracy: 0.22 Avg loss: 2.27



Validation Results - Epoch: 2  Avg accuracy: 0.46 Avg loss: 2.18



Validation Results - Epoch: 3  Avg accuracy: 0.69 Avg loss: 1.93



Validation Results - Epoch: 4  Avg accuracy: 0.77 Avg loss: 1.39



Validation Results - Epoch: 5  Avg accuracy: 0.83 Avg loss: 0.92


...


Validation Results - Epoch: 7  Avg accuracy: 0.88 Avg loss: 0.52



Validation Results - Epoch: 8  Avg accuracy: 0.90 Avg loss: 0.44



Validation Results - Epoch: 9  Avg accuracy: 0.90 Avg loss: 0.38



Validation Results - Epoch: 10  Avg accuracy: 0.91 Avg loss: 0.34


State:
	iteration: 2350
	epoch: 10
	epoch_length: 235
	max_epochs: 10
	max_iters: <class 'NoneType'>
	output: 0.896207332611084
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

We can see the summary of the profiling results from our run using the `get_results()` method of the profiler as shown below. The output shows total, average and other details of execution time for each handler attached.

In [8]:
profiler.print_results(profiler.get_results())


--------------------------  -----------------------  --------------  --------------  --------------  --------------  --------------  
Handler                     Event Name                     Total(s)      Min(s)/IDX      Max(s)/IDX         Mean(s)          Std(s)  
--------------------------  -----------------------  --------------  --------------  --------------  --------------  --------------  
ProgressBar._close          EPOCH_COMPLETED                 0.01591       0.00096/3       0.00189/8         0.00159          0.0003  
log_validation_results      EPOCH_COMPLETED                10.90327       1.05686/5       1.14857/0         1.09033         0.02953  
_OutputHandler              ITERATION_COMPLETED             2.93056    0.00013/1098    0.02848/1175         0.00125         0.00152  
--------------------------  -----------------------  --------------  --------------  --------------  --------------  --------------  
Total                                                      13

/home/perceptron/git-repos/ignite/ignite/contrib/handlers/time_profilers.py:587: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data, dtype=torch.float32)


Profiling results can be exported to a CSV file by using the `write_results()` method of profiler.

In [9]:
profiler.write_results("./results.csv")

Following code shows the preview of few rows of the CSV. Each handler has its separate column and the numbers of rows for each column will be equal to the number of times the handler invoked.

In [10]:
import pandas as pd

In [11]:
results = pd.read_csv("./results.csv")
results.head()

,#,processing_stats,dataflow_stats,ProgressBar._close (EPOCH_COMPLETED),log_validation_results (EPOCH_COMPLETED),_OutputHandler (ITERATION_COMPLETED)
0,1.0,0.037544,0.051581,0.001696,1.148570,0.022729
1,2.0,0.005099,0.003080,0.001808,1.099958,0.000911
2,3.0,0.005343,0.000208,0.001319,1.086610,0.001547
3,4.0,0.006225,0.000205,0.000963,1.111351,0.001826
4,5.0,0.004996,0.005869,0.001816,1.096900,0.001447
